# Fast Inference Test: Qwen3-4B

Tests `fast_inference=True` with vLLM backend on Qwen3-4B.

**Important:** This notebook includes a kernel shutdown cell at the end.
vLLM does not release GPU memory in single-process mode (Jupyter), so kernel
restart is required between different model tests.

In [1]:
# Environment Setup (quiet mode)
import warnings
import os
import sys
import logging

# Suppress all verbose output
warnings.filterwarnings("ignore")
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["TQDM_DISABLE"] = "1"
logging.getLogger("unsloth").setLevel(logging.ERROR)
logging.getLogger("vllm").setLevel(logging.ERROR)
logging.getLogger("transformers").setLevel(logging.ERROR)

from dotenv import load_dotenv
load_dotenv()

# Suppress unsloth banner during import
from contextlib import redirect_stdout, redirect_stderr
from io import StringIO
with redirect_stdout(StringIO()), redirect_stderr(StringIO()):
    import unsloth
    from unsloth import FastLanguageModel

import vllm
import torch

# Suppress model loading verbosity
from transformers import logging as hf_logging
hf_logging.set_verbosity_error()

# Single-line environment summary
gpu = torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU"
print(f"Environment: unsloth {unsloth.__version__}, vLLM {vllm.__version__}, {gpu}")

HF_TOKEN loaded: Yes🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.

/opt/pixi/.pixi/envs/default/lib/python3.13/site-packages/trl/__init__.py:203: UserWarning: TRL currently supports vLLM versions: 0.10.2, 0.11.0, 0.11.1, 0.11.2. You have version 0.14.0rc1.dev201+gadcf682fc.cu130 installed. We recommend installing a supported version to avoid compatibility issues.
  if is_vllm_available():

🦥 Unsloth Zoo will now patch everything to make training faster!unsloth: 2025.12.10
transformers: 5.0.0rc1
vLLM: 0.14.0rc1.dev201+gadcf682fc
TRL: 0.26.2
PyTorch: 2.9.1+cu130
CUDA: True
GPU: NVIDIA GeForce RTX 4080 SUPER

In [2]:
# Test Qwen3-4B with fast_inference=True
MODEL_NAME = "unsloth/Qwen3-4B-unsloth-bnb-4bit"
print(f"\nTesting {MODEL_NAME.split('/')[-1]} with fast_inference=True...")

from vllm import SamplingParams
import time
import os

# Suppress verbose model loading output by redirecting to /dev/null
_stdout_fd = os.dup(1)
_stderr_fd = os.dup(2)
_devnull = os.open(os.devnull, os.O_WRONLY)
os.dup2(_devnull, 1)
os.dup2(_devnull, 2)

try:
    model, tokenizer = FastLanguageModel.from_pretrained(
        MODEL_NAME,
        max_seq_length=512,
        load_in_4bit=True,
        fast_inference=True,
    )
finally:
    os.dup2(_stdout_fd, 1)
    os.dup2(_stderr_fd, 2)
    os.close(_devnull)
    os.close(_stdout_fd)
    os.close(_stderr_fd)

# Test generation
FastLanguageModel.for_inference(model)
messages = [{"role": "user", "content": "Say hello in one word."}]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
sampling_params = SamplingParams(temperature=0.1, max_tokens=10)

start = time.time()
outputs = model.fast_generate([prompt], sampling_params=sampling_params)
elapsed = time.time() - start

# Clear result
print(f"\n{'='*60}")
print(f"Model: {MODEL_NAME}")
print(f"FastInference: ✅ SUPPORTED")
print(f"Generation: {elapsed:.2f}s")
print(f"{'='*60}")

=== Qwen3-4B Fast Inference Test (vLLM Backend) ===
Testing cross-architecture fast_inference support...WARNING 01-02 23:11:47 [vllm.py:1427] Current vLLM config is not set.INFO 01-02 23:11:47 [scheduler.py:231] Chunked prefill is enabled with max_num_batched_tokens=2048.INFO 01-02 23:11:47 [vllm.py:609] Disabling NCCL for DP synchronization when using async scheduling.INFO 01-02 23:11:47 [vllm.py:614] Asynchronous scheduling is enabled.INFO 01-02 23:11:47 [vllm_utils.py:702] Unsloth: Patching vLLM v1 graph capture==((====))==  Unsloth 2025.12.10: Fast Qwen3 patching. Transformers: 5.0.0.1. vLLM: 0.14.0rc1.dev201+gadcf682fc.cu130.
   \\   /|    NVIDIA GeForce RTX 4080 SUPER. Num GPUs = 1. Max memory: 15.568 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu130. CUDA: 8.9. CUDA Toolkit: 13.0. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore 

/opt/pixi/.pixi/envs/default/lib/python3.13/site-packages/pydantic/type_adapter.py:605: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `enum` - serialized value may not be as expected [field_name='mode', input_value=3, input_type=int])
  return self.serializer.to_python(

INFO 01-02 23:11:55 [model.py:517] Resolved architecture: Qwen3ForCausalLMINFO 01-02 23:11:55 [model.py:1688] Using max model len 512WARNING 01-02 23:11:55 [vllm.py:1427] Current vLLM config is not set.INFO 01-02 23:11:55 [scheduler.py:231] Chunked prefill is enabled with max_num_batched_tokens=2048.INFO 01-02 23:11:55 [scheduler.py:231] Chunked prefill is enabled with max_num_batched_tokens=2048.Unsloth: vLLM Bitsandbytes config using kwargs = {'load_in_8bit': False, 'load_in_4bit': True, 'bnb_4bit_compute_dtype': 'bfloat16', 'bnb_4bit_quant_storage': 'uint8', 'bnb_4bit_quant_type': 'nf4', 'bnb_4bit_use_double_quant': True, 'llm_int8_enable_fp32_cpu_offload': False, 'llm_int8_has_fp16_weight': False, 'llm_int8_skip_modules': ['lm_head', 'multi_modal_projector', 'merger', 'modality_projection', 'model.layers.4.mlp', 'model.layers.6.self_attn', 'model.layers.34.self_attn', 'model.layers.33.self_attn', 'model.layers.4.self_attn', 'model.layers.34.mlp', 'model.layers.1.self_attn', 'model.

/opt/pixi/.pixi/envs/default/lib/python3.13/site-packages/pydantic/type_adapter.py:605: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `enum` - serialized value may not be as expected [field_name='mode', input_value=3, input_type=int])
  return self.serializer.to_python(

INFO 01-02 23:11:57 [topk_topp_sampler.py:47] Using FlashInfer for top-p & top-k sampling.INFO 01-02 23:11:57 [gpu_model_runner.py:3762] Starting to load model unsloth/Qwen3-4B-unsloth-bnb-4bit...INFO 01-02 23:11:58 [cuda.py:315] Using AttentionBackendEnum.FLASHINFER backend.INFO 01-02 23:11:58 [bitsandbytes_loader.py:790] Loading weights with BitsAndBytes quantization. May take a while ...INFO 01-02 23:11:59 [weight_utils.py:550] No model.safetensors.index.json found in remote.

...

Loading weights:   0%|          | 0/398 [00:00<?, ?it/s]

Performing substitution for additional_keys=set()
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'pre_feedforward_layernorm', 'cross_attn_input_layernorm', 'post_layernorm', 'input_layernorm', 'norm1', 'post_attention_layernorm', 'norm', 'layer_norm2', 'ffn_norm', 'norm2', 'cross_attn_post_attention_layernorm', 'k_norm', 'attention_norm', 'q_norm', 'layer_norm1']✓ Qwen3-4B loaded with fast_inference=True

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

✓ vLLM generation completed in 0.10s
  Response: 
Okay, the user asked me to say
✓ Qwen3-4B fast_inference test PASSED

## Test Complete

The Qwen3-4B fast_inference test has completed. The kernel will now shut down to release all GPU memory.

**Next:** Run `03_FastInference_Ministral.ipynb` for Ministral model testing.

In [3]:
# Shutdown kernel to release all GPU memory
import IPython
print("Shutting down kernel to release GPU memory...")
app = IPython.Application.instance()
app.kernel.do_shutdown(restart=False)

Shutting down kernel to release GPU memory...

{'status': 'ok', 'restart': False}